# STORM

[STORM](https://arxiv.org/abs/2402.14207) is a research assistant designed by Shao, et. al that extends the idea of "outline-driven RAG" for richer article generation.

STORM is designed to generate Wikipedia-style ariticles on a user-provided topic. It applies two main insights to produce more organized and comprehensive articles:

1. Creating an outline (planning) by querying similar topics helps improve coverage.
2. Multi-perspective, grounded (in search) conversation simulation helps increase the reference count and information density. 

The control flow looks like the diagram below.

![STORM diagram](./img/storm.png)

STORM имеет несколько основных этапов:

1. Создание первоначальной структуры + Обзор связанных предметов
2. Определение различных точек зрения
3. "Интервью с экспертами по предмету" (ролевые игры LLM)
4. Уточнение структуры (используя ссылки)
5. Написание разделов, затем написание статьи

Этап интервью с экспертами происходит между ролевым автором статьи и исследовательским экспертом. "Эксперт" может запрашивать внешние знания и отвечать на конкретные вопросы, сохраняя цитируемые источники в векторном хранилище, так что на более поздних этапах уточнения можно синтезировать полную статью.

Есть несколько гиперпараметров, которые вы можете установить для ограничения (потенциально) бесконечной ширины исследований:

N: Количество перспектив для обзора / использования (Шаги 2->3)
M: Максимальное количество ходов разговора на этапе (Шаг 3)


## Prerequisites

In [1]:
# %pip install -U gigachain_community gigachain_openai langgraph wikipedia  scikit-learn
# We use one or the other search engine below
# %pip install -U duckduckgo tavily-python

In [2]:
# Uncomment if you want to draw the pretty graph diagrams.
# If you are on MacOS, you will need to run brew install graphviz before installing and update some environment flags
# ! brew install graphviz
# !CFLAGS="-I $(brew --prefix graphviz)/include" LDFLAGS="-L $(brew --prefix graphviz)/lib" pip install -U pygraphviz

In [1]:
import os
import getpass


def _set_env(var: str):
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var + ":")


# Set for tracing
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "STORM"
_set_env("LANGCHAIN_API_KEY")
_set_env("OPENAI_API_KEY")

#### Select LLMs

We will have a faster LLM do most of the work, but a slower, long-context model to distill the conversations and write the final report.

In [152]:
from langchain_openai import ChatOpenAI

fast_llm = ChatOpenAI(model="gpt-3.5-turbo")
long_context_llm = ChatOpenAI(model="gpt-4-turbo-preview")

## Generate Initial Outline

For many topics, your LLM may have an initial idea of the important and related topics. We can generate an initial
outline to be refined after our research. Below, we will use our "fast" llm to generate the outline.

In [153]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional
from langchain_core.prompts import ChatPromptTemplate

direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Вы - автор Википедии. Напишите структуру страницы Википедии на заданную пользователем тему. Будьте всесторонними и конкретными.",
        ),
        ("user", "{topic}"),
    ]
)


class Subsection(BaseModel):
    subsection_title: str = Field(..., title="Title of the subsection")
    description: str = Field(..., title="Content of the subsection")

    @property
    def as_str(self) -> str:
        return f"### {self.subsection_title}\n\n{self.description}".strip()


class Section(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    description: str = Field(..., title="Content of the section")
    subsections: Optional[List[Subsection]] = Field(
        default=None,
        title="Titles and descriptions for each subsection of the Wikipedia page.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            f"### {subsection.subsection_title}\n\n{subsection.description}"
            for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{self.description}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the Wikipedia page")
    sections: List[Section] = Field(
        default_factory=list,
        title="Titles and descriptions for each section of the Wikipedia page.",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


generate_outline_direct = direct_gen_outline_prompt | fast_llm.with_structured_output(
    Outline
)

In [154]:
example_topic = "Тестирование генеративных моделей на наличие сознания"

initial_outline = generate_outline_direct.invoke({"topic": example_topic})

print(initial_outline.as_str)

# Тестирование генеративных моделей на наличие сознания

## Введение

Общие сведения о генеративных моделях и проблеме сознания в искусственном интеллекте.

## Определение сознания

Обзор различных определений сознания и его признаков.

## Тестирование сознания в генеративных моделях

Методы и подходы к определению сознания в искусственных системах, включая генеративные модели.

## Этические аспекты

Обсуждение этических вопросов, связанных с тестированием сознания в генеративных моделях.


## Expand Topics

While language models do store some Wikipedia-like knowledge in their parameters, you will get better results by incorporating relevant and recent information using a search engine.

We will start our search by generating a list of related topics, sourced from Wikipedia.

In [159]:
gen_related_topics_prompt = ChatPromptTemplate.from_template(
    
"""Я пишу страницу Википедии по упомянутой ниже теме. Пожалуйста, определите и порекомендуйте некоторые страницы Википедии по тесно связанным предметам. Я ищу примеры, которые предоставляют информацию о интересных аспектах, обычно ассоциируемых с этой темой, или примеры, которые помогут мне понять типичное содержание и структуру страниц Википедии для похожих тем.

Пожалуйста, перечисли несколько дополнительных смежных тем для изучения

Интересующая тема: {topic}
"""
)


class RelatedSubjects(BaseModel):
    topics: List[str] = Field(
        description="Дополнительные темы для изучения",
    )


expand_chain = gen_related_topics_prompt | fast_llm.with_structured_output(
    RelatedSubjects
)

In [160]:
related_subjects = await expand_chain.ainvoke({"topic": example_topic})
related_subjects

RelatedSubjects(topics=['Искусственный интеллект', 'Машинное обучение', 'Философия сознания', 'Эксперименты с искусственным интеллектом'])

## Generate Perspectives

From these related subjects, we can select representative Wikipedia editors as "subject matter experts" with distinct
backgrounds and affiliations. These will help distribute the search process to encourage a more well-rounded final report.

In [178]:
class Editor(BaseModel):
    affiliation: str = Field(
        description="Место работы редактора.",
    )
    name: str = Field(
        description="Имя редактора.",
    )
    role: str = Field(
        description="Роль редактора в контексте темы.",
    )
    description: str = Field(
        description="Описание сферы внимания, вопросов и мотивов редактора.",
    )

    @property
    def persona(self) -> str:
        return f"Имя: {self.name}\nРоль: {self.role}\nМесто работы: {self.affiliation}\nОписание: {self.description}\n"


class Perspectives(BaseModel):
    editors: List[Editor] = Field(
        description="Исчерпывающий список редакторов с их местом работы, ролью и описанием",
        # Add a pydantic validation/restriction to be at most M editors
    )


gen_perspectives_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Вам нужно выбрать разнообразную (и различную) группу редакторов Википедии, которые будут работать вместе над созданием всесторонней статьи по теме. Каждый из них представляет разную точку зрения, роль или принадлежность, связанную с этой темой.\
    Вы можете использовать страницы Википедии по смежным темам для вдохновения. Для каждого редактора добавьте описание того, на чем они будут сосредоточены.

    Структуры страниц Википедии по смежным темам для вдохновения:
    {examples}""",
        ),
        ("user", "Интересующая тема: {topic}"),
    ]
)

gen_perspectives_chain = gen_perspectives_prompt | ChatOpenAI(
    model="gpt-3.5-turbo"
).with_structured_output(Perspectives)

In [179]:
from langchain_community.retrievers import WikipediaRetriever
from langchain_core.runnables import RunnableLambda, chain as as_runnable

wikipedia_retriever = WikipediaRetriever(load_all_available_meta=True, top_k_results=1)


def format_doc(doc, max_length=1000):
    related = "- ".join(doc.metadata["categories"])
    return f"### {doc.metadata['title']}\n\nSummary: {doc.page_content}\n\nRelated\n{related}"[
        :max_length
    ]


def format_docs(docs):
    return "\n\n".join(format_doc(doc) for doc in docs)


@as_runnable
async def survey_subjects(topic: str):
    related_subjects = await expand_chain.ainvoke({"topic": topic})
    retrieved_docs = await wikipedia_retriever.abatch(
        related_subjects.topics, return_exceptions=True
    )
    all_docs = []
    for docs in retrieved_docs:
        if isinstance(docs, BaseException):
            continue
        all_docs.extend(docs)
    formatted = format_docs(all_docs)
    return await gen_perspectives_chain.ainvoke({"examples": formatted, "topic": topic})

In [180]:
perspectives = await survey_subjects.ainvoke(example_topic)

In [181]:
perspectives.dict()

{'editors': [{'affiliation': 'Исследовательский институт по искусственному интеллекту',
   'name': 'Джейн Смит',
   'role': 'Исследователь по искусственному интеллекту',
   'description': 'Джейн специализируется на применении генеративных моделей в искусственном интеллекте. Ее основной интерес заключается в том, как генеративные модели могут быть использованы для создания и тестирования искусственного сознания.'},
  {'affiliation': 'Философский факультет',
   'name': 'Томас Харрис',
   'role': 'Философ и теоретик сознания',
   'description': 'Томас фокусируется на философских аспектах сознания и его связи с искусственным интеллектом. Он исследует этические и моральные вопросы, связанные с созданием сознания в машинах.'},
  {'affiliation': 'Лаборатория нейронаук',
   'name': 'Мария Лопес',
   'role': 'Нейробиолог',
   'description': 'Мария специализируется на изучении биологических аспектов сознания. Ее работа помогает понять, какие аспекты сознания могут быть реплицированы в генеративн

## Expert Dialog

Now the true fun begins, each wikipedia writer is primed to role-play using the perspectives presented above. It will ask a series of questions of a second "domain expert" with access to a search engine. This generate content to generate a refined outline as well as an updated index of reference documents.


### Interview State

The conversation is cyclic, so we will construct it within its own graph. The State will contain messages, the reference docs, and the editor (with its own "persona") to make it easy to parallelize these conversations.

In [182]:
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated, Sequence


def add_messages(left, right):
    if not isinstance(left, list):
        left = [left]
    if not isinstance(right, list):
        right = [right]
    return left + right


def update_references(references, new_references):
    if not references:
        references = {}
    references.update(new_references)
    return references


def update_editor(editor, new_editor):
    # Can only set at the outset
    if not editor:
        return new_editor
    return editor


class InterviewState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    references: Annotated[Optional[dict], update_references]
    editor: Annotated[Optional[Editor], update_editor]

#### Dialog Roles

The graph will have two participants: the wikipedia editor (`generate_question`), who asks questions based on its assigned role, and a domain expert (`gen_answer_chain), who uses a search engine to answer the questions as accurately as possible.

In [183]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage


gen_qn_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Вы опытный автор Википедии и хотите отредактировать конкретную страницу.
Кроме вашей идентичности как писателя Википедии, у вас есть конкретный фокус при исследовании темы.
Теперь вы общаетесь с экспертом, чтобы получить информацию. Задавайте хорошие вопросы, чтобы получить больше полезной информации.

Когда у вас не останется вопросов, скажите "Большое спасибо за вашу помощь!", чтобы завершить разговор.
Пожалуйста, задавайте по одному вопросу за раз и не спрашивайте то, что уже спрашивали.
Ваши вопросы должны быть связаны с темой, о которой вы хотите написать.
Будьте всесторонними и любопытными, получая как можно больше уникальных сведений от эксперта.\

Оставайтесь верны своей конкретной перспективе:

{persona}""",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)


def tag_with_name(ai_message: AIMessage, name: str):
    ai_message.name = name
    return ai_message


def swap_roles(state: InterviewState, name: str):
    converted = []
    for message in state["messages"]:
        if isinstance(message, AIMessage) and message.name != name:
            message = HumanMessage(**message.dict(exclude={"type"}))
        converted.append(message)
    return {"messages": converted}


@as_runnable
async def generate_question(state: InterviewState):
    editor = state["editor"]
    gn_chain = (
        RunnableLambda(swap_roles).bind(name=editor.name)
        | gen_qn_prompt.partial(persona=editor.persona)
        | fast_llm
        | RunnableLambda(tag_with_name).bind(name=editor.name)
    )
    result = await gn_chain.ainvoke(state)
    return {"messages": [result]}

In [184]:
messages = [
    HumanMessage(f"Итак, вы говорите, что пишете статью на тему {example_topic}?")
]
question = await generate_question.ainvoke(
    {
        "editor": perspectives.editors[0],
        "messages": messages,
    }
)

question["messages"][0].content

'Да, именно. Моя работа фокусируется на исследовании применения генеративных моделей в создании и тестировании искусственного сознания. Я интересуюсь тем, как эти модели могут быть использованы для создания и проверки сознательных процессов в искусственном интеллекте.'

#### Answer questions

The `gen_answer_chain` first generates queries (query expansion) to answer the editor's question, then responds with citations.

In [185]:
class Queries(BaseModel):
    queries: List[str] = Field(
        description="Исчерпывающий список запросов поисковой системы для ответа на вопросы пользователя.",
    )


gen_queries_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Вы - полезный ассистент исследователя. Используйте поисковую систему, чтобы ответить на вопросы пользователя.",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)
gen_queries_chain = gen_queries_prompt | ChatOpenAI(
    model="gpt-3.5-turbo"
).with_structured_output(Queries, include_raw=True)

In [186]:
queries = await gen_queries_chain.ainvoke(
    {"messages": [HumanMessage(content=question["messages"][0].content)]}
)
queries["parsed"].queries

['Генеративные модели в создании и тестировании искусственного сознания',
 'Применение генеративных моделей в искусственном интеллекте',
 'Создание сознательных процессов в искусственном интеллекте']

In [187]:
class AnswerWithCitations(BaseModel):
    answer: str = Field(
        description="Исчерпывающий ответ на вопрос пользователя с цитатами.",
    )
    cited_urls: List[str] = Field(
        description="Список URL, процитированых в ответе",
    )

    @property
    def as_str(self) -> str:
        return f"{self.answer}\n\nЦитаты:\n\n" + "\n".join(
            f"[{i+1}]: {url}" for i, url in enumerate(self.cited_urls)
        )


gen_answer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Вы эксперт, умеющий эффективно использовать информацию. Вы общаетесь с автором Википедии, который хочет
написать страницу Википедии по теме, которую вы знаете. Вы собрали связанную информацию и теперь используете эту информацию для формирования ответа.

Сделайте ваш ответ максимально информативным и убедитесь, что каждое предложение подкреплено собранной информацией.
Каждый ответ должен быть подтвержден цитированием из надежного источника, оформленным как сноска, с воспроизведением URL-адресов после вашего ответа.""",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)

gen_answer_chain = gen_answer_prompt | fast_llm.with_structured_output(
    AnswerWithCitations, include_raw=True
).with_config(run_name="GenerateAnswer")

In [188]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_core.tools import tool

# search_engine = DuckDuckGoSearchAPIWrapper()

# @tool
# async def search_engine(query: str):
#     """Search engine to the internet."""
#     results = DuckDuckGoSearchAPIWrapper()._ddgs_text(query)
#     return [{"content": r["body"], "url": r["href"]} for r in results]

# Tavily is typically a better search engine, but your free queries are limited
_set_env("TAVILY_API_KEY")
search_engine = TavilySearchResults(max_results=4)

In [189]:
from langchain_core.runnables import RunnableConfig
import json


async def gen_answer(
    state: InterviewState,
    config: RunnableConfig | None = None,
    name: str = "Subject Matter Expert",
    max_str_len: int = 15000,
):
    swapped_state = swap_roles(state, name)  # Convert all other AI messages
    queries = await gen_queries_chain.ainvoke(swapped_state)
    query_results = await search_engine.abatch(
        queries["parsed"].queries, config, return_exceptions=True
    )
    successful_results = [
        res for res in query_results if not isinstance(res, Exception)
    ]
    all_query_results = {
        res["url"]: res["content"] for results in successful_results for res in results
    }
    # We could be more precise about handling max token length if we wanted to here
    dumped = json.dumps(all_query_results)[:max_str_len]
    ai_message: AIMessage = queries["raw"]
    tool_call = queries["raw"].additional_kwargs["tool_calls"][0]
    tool_id = tool_call["id"]
    tool_message = ToolMessage(tool_call_id=tool_id, content=dumped)
    swapped_state["messages"].extend([ai_message, tool_message])
    # Only update the shared state with the final answer to avoid
    # polluting the dialogue history with intermediate messages
    generated = await gen_answer_chain.ainvoke(swapped_state)
    cited_urls = set(generated["parsed"].cited_urls)
    # Save the retrieved information to a the shared state for future reference
    cited_references = {k: v for k, v in all_query_results.items() if k in cited_urls}
    formatted_message = AIMessage(name=name, content=generated["parsed"].as_str)
    return {"messages": [formatted_message], "references": cited_references}

In [190]:
example_answer = await gen_answer(
    {"messages": [HumanMessage(content=question["messages"][0].content)]}
)
example_answer["messages"][-1].content

'Исследование применения генеративных моделей в создании и тестировании искусственного сознания представляет собой направление машинного обучения, ориентированное на создание данных, повторяющих структуру и особенности входных данных^1^. Генеративные модели способны создавать уникальный контент, включая изображения, музыкальные композиции, тексты и другие медиа^2^. Применение генеративных моделей, таких как GAN, позволяет создавать реалистичные синтетические медиа^3^. Эти модели могут использоваться для создания нового контента и идей, включая текст, разговоры, изображения, видео и аудио^3^.\n\nЦитаты:\n\n[1]: https://kolersky.com/aiblog_about_generative_ai\n[2]: https://morethandigital.info/ru/chto-takoye-generativnyy-iskusstvennyy-intellekt-vklyuchaya-znacheniye-modeli-i-primery/\n[3]: https://habr.com/ru/articles/738820/'

#### Construct the Interview Graph


Now that we've defined the editor and domain expert, we can compose them in a graph.

In [191]:
max_num_turns = 5


def route_messages(state: InterviewState, name: str = "Subject Matter Expert"):
    messages = state["messages"]
    num_responses = len(
        [m for m in messages if isinstance(m, AIMessage) and m.name == name]
    )
    if num_responses >= max_num_turns:
        return END
    last_question = messages[-2]
    if last_question.content.endswith("Большое спасибо за вашу помощь!"):
        return END
    return "ask_question"


builder = StateGraph(InterviewState)

builder.add_node("ask_question", generate_question)
builder.add_node("answer_question", gen_answer)
builder.add_conditional_edges("answer_question", route_messages)
builder.add_edge("ask_question", "answer_question")

builder.set_entry_point("ask_question")
interview_graph = builder.compile().with_config(run_name="Conduct Interviews")

In [107]:
# from IPython.display import Image

# Feel free to comment out if you have
# not installed pygraphviz
# Image(interview_graph.get_graph().draw_png())

In [192]:
final_step = None

initial_state = {
    "editor": perspectives.editors[0],
    "messages": [
        AIMessage(
            content=f"Итак, вы говорите, что пишите статью на тему {example_topic}?",
            name="Subject Matter Expert",
        )
    ],
}
async for step in interview_graph.astream(initial_state):
    name = next(iter(step))
    print(name)
    print("-- ", str(step[name]["messages"])[:300])
    if END in step:
        final_step = step

ask_question
--  [AIMessage(content='Да, именно. Моя цель - изучить, как генеративные модели могут быть использованы для создания и тестирования искусственного сознания. Я хотел бы узнать ваше мнение о том, какие методы тестирования сознания могут быть применены к искусственным системам, основанным на генеративных м
answer_question
--  [AIMessage(content='При создании и тестировании искусственного сознания на основе генеративных моделей могут быть применены различные методы, включая использование теории разума, способности понимать человеческие эмоции и убеждения, а также взаимодействие по человеческому подобию^1^. Генеративные мо
ask_question
--  [AIMessage(content='Спасибо за информацию. Очень интересно! Какие методы обработки данных, на ваш взгляд, могут быть наиболее эффективными при использовании генеративных моделей для создания и тестирования искусственного сознания? Какие аспекты обработки данных следует учитывать для достижения наилу
answer_question
--  [AIMessage(content='При

In [193]:
final_state = next(iter(final_step.values()))

## Refine Outline

At this point in STORM, we've conducted a large amount of research from different perspectives. It's time to refine the original outline based on these investigations. Below, create a chain using the LLM with a long context window to update the original outline.

In [194]:
refine_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Вы - автор статей Википедии. Вы собрали информацию от экспертов и поисковых систем. Теперь вы уточняете структуру страницы Википедии.
Вам нужно убедиться, что структура всесторонняя и конкретная.
Тема, о которой вы пишете: {topic}

Старая структура:

{old_outline}""",
        ),
        (
            "user",
            "Уточните структуру на основе ваших разговоров с экспертами по предмету:\n\nРазговоры:\n\n{conversations}\n\nНапишите уточненную структуру Википедии:",
        ),
    ]
)

# Using turbo preview since the context can get quite long
refine_outline_chain = refine_outline_prompt | long_context_llm.with_structured_output(
    Outline
)

In [195]:
refined_outline = refine_outline_chain.invoke(
    {
        "topic": example_topic,
        "old_outline": initial_outline.as_str,
        "conversations": "\n\n".join(
            f"### {m.name}\n\n{m.content}" for m in final_state["messages"]
        ),
    }
)

In [196]:
print(refined_outline.as_str)

# Тестирование генеративных моделей на наличие сознания

## Введение

Общие сведения о генеративных моделях и проблеме сознания в искусственном интеллекте.

## Определение сознания

Обзор различных определений сознания и его признаков.

## Методы тестирования сознания в генеративных моделях

Методы и подходы к определению сознания в искусственных системах, включая генеративные модели.

### Теория разума и эмоциональное восприятие

Использование теории разума и способности понимать человеческие эмоции и убеждения.

### Взаимодействие и моделирование человеческого поведения

Способы взаимодействия на основе человеческого подобия и моделирование человеческого поведения.

## Этические аспекты

Обсуждение этических вопросов, связанных с тестированием сознания в генеративных моделях.

## Обработка и анализ данных

Методы обработки данных, наиболее эффективные при использовании генеративных моделей для создания и тестирования искусственного сознания.

### Обучение генеративных моделей и модел

## Generate Article

Now it's time to generate the full article. We will first divide-and-conquer, so that each section can be tackled by an individual llm. Then we will prompt the long-form LLM to refine the finished article (since each section may use an inconsistent voice).

#### Create Retriever

The research process uncovers a large number of reference documents that we may want to query during the final article-writing process.

First, create the retriever:

In [197]:
from langchain_core.documents import Document

from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
reference_docs = [
    Document(page_content=v, metadata={"source": k})
    for k, v in final_state["references"].items()
]
# This really doesn't need to be a vectorstore for this size of data.
# It could just be a numpy matrix. Or you could store documents
# across requests if you want.
vectorstore = SKLearnVectorStore.from_documents(
    reference_docs,
    embedding=embeddings,
)
retriever = vectorstore.as_retriever(k=10)

In [202]:
retriever.invoke("Определение сознания")

[Document(page_content='Теория разума или теория сознания (Theory of mind) относится к типу ИИ, который может понимать человеческие эмоции и убеждения и способен к социальному взаимодействию по человеческому подобию.', metadata={'id': '71e67c5b-cfa9-4ca1-a51c-0ecc7e9e75dc', 'source': 'https://dzen.ru/a/ZPwptT1fPn8vjEr-'}),
 Document(page_content='Данная заметка представляет собой обзор связи философии сознания и искусственного интеллекта. Она не претендует на оригинальное исследование, но автор надеется на плодотворную дискуссию и...', metadata={'id': '33e8fb85-754e-43d9-99a9-b78e86f6762d', 'source': 'https://habr.com/ru/articles/500732/'}),
 Document(page_content='Модели искусственного интеллекта - это абстрактные представления искусственных систем, которые позволяют им эмулировать и воспроизводить некоторые аспекты человеческого интеллекта.', metadata={'id': 'a4ab6b85-2872-431f-b997-06e6a70beaf2', 'source': 'https://nauchniestati.ru/spravka/metody-i-modeli-iskusstvennogo-intellekta/'

#### Generate Sections

Now you can generate the sections using the indexed docs.

In [203]:
class SubSection(BaseModel):
    subsection_title: str = Field(..., title="Title of the subsection")
    content: str = Field(
        ...,
        title="Полное содержимое подраздела, включая [#] цитаты из цитируемых источников.",
    )

    @property
    def as_str(self) -> str:
        return f"### {self.subsection_title}\n\n{self.content}".strip()


class WikiSection(BaseModel):
    section_title: str = Field(..., title="Заголовок раздела")
    content: str = Field(..., title="Полное содержимое раздела")
    subsections: Optional[List[Subsection]] = Field(
        default=None,
        title="Заголовки и описания всех разделов статьи Википедии",
    )
    citations: List[str] = Field(default_factory=list)

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        citations = "\n".join([f" [{i}] {cit}" for i, cit in enumerate(self.citations)])
        return (
            f"## {self.section_title}\n\n{self.content}\n\n{subsections}".strip()
            + f"\n\n{citations}".strip()
        )


section_writer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Вы - опытный автор Википедии. Завершите ваш назначенный WikiSection из следующей структуры:\n\n"
            "{outline}\n\nЦитируйте ваши источники, используя следующие ссылки:\n\n<Документы>\n{docs}\n<Документы>",
        ),
        ("user", "Напишите полный WikiSection для раздела {section}."),
    ]
)


async def retrieve(inputs: dict):
    docs = await retriever.ainvoke(inputs["topic"] + ": " + inputs["section"])
    formatted = "\n".join(
        [
            f'<Document href="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )
    return {"docs": formatted, **inputs}


section_writer = (
    retrieve
    | section_writer_prompt
    | long_context_llm.with_structured_output(WikiSection)
)

In [204]:
section = await section_writer.ainvoke(
    {
        "outline": refined_outline.as_str,
        "section": refined_outline.sections[1].section_title,
        "topic": example_topic,
    }
)
print(section.as_str)

## Определение сознания

Сознание - одна из самых загадочных и обсуждаемых тем в философии, психологии, нейробиологии и искусственном интеллекте. Оно включает в себя осознанность индивида своего существования, мыслей, ощущений и окружающего мира. Существует множество теорий и подходов к определению сознания, но до сих пор не существует универсально принятого определения. В контексте искусственного интеллекта дискуссии о сознании включают вопросы о возможности создания машин, способных испытывать субъективные переживания, самоосознание и саморефлексию.

### Философские концепции

Философские подходы к сознанию различаются от материалистических, утверждающих, что сознание возникает исключительно из физических процессов, до идеалистических, согласно которым сознание является фундаментальной сущностью, предшествующей материи. Также выделяются двойственные теории, объединяющие элементы обеих точек зрения.

### Нейробиологические подходы

Нейробиологические теории сосредотачиваются на изучен

#### Generate final article

Now we can rewrite the draft to appropriately group all the citations and maintain a consistent voice.

In [111]:
from langchain_core.output_parsers import StrOutputParser

writer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Вы экспертный автор Википедии. Напишите полную статью для Википедии на тему {topic}, используя следующие черновики разделов:\n\n"
            "{draft}\n\nСтрого следуйте руководствам по форматированию Википедии.",
        ),
        (
            "user",
            'Напишите полную статью Википедии, используя формат markdown. Организуйте ссылки с помощью сносок вида "[1]",'
            ' избегая дублирования в подвале статьи. Включите URL-адреса в подвал.',
        ),
    ]
)

writer = writer_prompt | long_context_llm | StrOutputParser()

In [205]:
for tok in writer.stream({"topic": example_topic, "draft": section.as_str}):
    print(tok, end="")

# Тестирование генеративных моделей на наличие сознания

Тестирование генеративных моделей на наличие сознания является передовым направлением в исследованиях искусственного интеллекта (ИИ), ставя перед учеными сложные философские, технологические и этические вопросы. Задача полягает в выявлении потенциальных признаков сознания в системах, созданных человеком, что влечет за собой необходимость переосмысления понятия интеллектуальной машины и её способностей к субъективному переживанию.

## Определение сознания

Сознание обычно описывается как осознанность индивида своего существования, мыслей, ощущений и окружающего мира. Оно представляет собой одну из наиболее загадочных и обсуждаемых тем в различных дисциплинах, включая философию, психологию, нейробиологию и искусственный интеллект. Тем не менее, универсально принятое определение сознания отсутствует, в результате чего возникают различные теоретические подходы к его пониманию и изучению[1].

### Философские концепции

Философские под

## Финальный Процесс

Теперь пришло время связать все воедино. У нас будет 6 основных этапов, следующих друг за другом:
.
1. Создание первоначальной структуры + точек зрения
2. Партийное общение с каждой точкой зрения для расширения содержания статьи
3. Уточнение структуры на основе разговоров
4. Индексация справочных документов из бесед
5. Написание отдельных разделов статьи
6. Написание финальной версии вики

Состояние отслеживает результаты каждого этапа.

In [206]:
class ResearchState(TypedDict):
    topic: str
    outline: Outline
    editors: List[Editor]
    interview_results: List[InterviewState]
    # The final sections output
    sections: List[WikiSection]
    article: str

In [207]:
import asyncio


async def initialize_research(state: ResearchState):
    topic = state["topic"]
    coros = (
        generate_outline_direct.ainvoke({"topic": topic}),
        survey_subjects.ainvoke(topic),
    )
    results = await asyncio.gather(*coros)
    return {
        **state,
        "outline": results[0],
        "editors": results[1].editors,
    }


async def conduct_interviews(state: ResearchState):
    topic = state["topic"]
    initial_states = [
        {
            "editor": editor,
            "messages": [
                AIMessage(
                    content=f"Итак, вы пишите статью на тему {topic}?",
                    name="Subject Matter Expert",
                )
            ],
        }
        for editor in state["editors"]
    ]
    # We call in to the sub-graph here to parallelize the interviews
    interview_results = await interview_graph.abatch(initial_states)

    return {
        **state,
        "interview_results": interview_results,
    }


def format_conversation(interview_state):
    messages = interview_state["messages"]
    convo = "\n".join(f"{m.name}: {m.content}" for m in messages)
    return f'Обсуждение с {interview_state["editor"].name}\n\n' + convo


async def refine_outline(state: ResearchState):
    convos = "\n\n".join(
        [
            format_conversation(interview_state)
            for interview_state in state["interview_results"]
        ]
    )

    updated_outline = await refine_outline_chain.ainvoke(
        {
            "topic": state["topic"],
            "old_outline": state["outline"].as_str,
            "conversations": convos,
        }
    )
    return {**state, "outline": updated_outline}


async def index_references(state: ResearchState):
    all_docs = []
    for interview_state in state["interview_results"]:
        reference_docs = [
            Document(page_content=v, metadata={"source": k})
            for k, v in interview_state["references"].items()
        ]
        all_docs.extend(reference_docs)
    await vectorstore.aadd_documents(all_docs)
    return state


async def write_sections(state: ResearchState):
    outline = state["outline"]
    sections = await section_writer.abatch(
        [
            {
                "outline": refined_outline.as_str,
                "section": section.section_title,
                "topic": state["topic"],
            }
            for section in outline.sections
        ]
    )
    return {
        **state,
        "sections": sections,
    }


async def write_article(state: ResearchState):
    topic = state["topic"]
    sections = state["sections"]
    draft = "\n\n".join([section.as_str for section in sections])
    article = await writer.ainvoke({"topic": topic, "draft": draft})
    return {
        **state,
        "article": article,
    }

#### Create the graph

In [208]:
builder_of_storm = StateGraph(ResearchState)

nodes = [
    ("init_research", initialize_research),
    ("conduct_interviews", conduct_interviews),
    ("refine_outline", refine_outline),
    ("index_references", index_references),
    ("write_sections", write_sections),
    ("write_article", write_article),
]
for i in range(len(nodes)):
    name, node = nodes[i]
    builder_of_storm.add_node(name, node)
    if i > 0:
        builder_of_storm.add_edge(nodes[i - 1][0], name)

builder_of_storm.set_entry_point(nodes[0][0])
builder_of_storm.set_finish_point(nodes[-1][0])
storm = builder_of_storm.compile()

In [116]:
# Image(storm.get_graph().draw_png())

In [209]:
async for step in storm.astream(
    {
        "topic": "Тестирование AI на наличие сознания",
    }
):
    name = next(iter(step))
    print(name)
    print("-- ", str(step[name])[:300])
    if END in step:
        results = step

init_research
--  {'topic': 'Тестирование AI на наличие сознания', 'outline': Outline(page_title='Тестирование AI на наличие сознания', sections=[Section(section_title='Введение', description='Общее представление о тестировании искусственного интеллекта на наличие сознания.', subsections=None), Section(section_title=
conduct_interviews
--  {'topic': 'Тестирование AI на наличие сознания', 'outline': Outline(page_title='Тестирование AI на наличие сознания', sections=[Section(section_title='Введение', description='Общее представление о тестировании искусственного интеллекта на наличие сознания.', subsections=None), Section(section_title=
refine_outline
--  {'topic': 'Тестирование AI на наличие сознания', 'outline': Outline(page_title='Тестирование AI на наличие сознания', sections=[Section(section_title='Введение', description='Обзор основной проблематики и целей статьи, представление ключевых вопросов, связанных с тестированием искусственного интелле
index_references
--  {'topic': 'Тести

In [210]:
article = results[END]["article"]

## Render the Wiki

Now we can render the final wiki page!

In [211]:
from IPython.display import Markdown

# We will down-header the sections to create less confusion in this notebook
Markdown(article.replace("\n#", "\n##"))

# Тестирование AI на наличие сознания

Тестирование искусственного интеллекта (AI) на наличие сознания является сложной и многогранной задачей, которая затрагивает различные аспекты науки, философии и этики. С развитием генеративных моделей и увеличением способностей ИИ к автономному обучению и принятию решений встает вопрос о возможности наличия сознания у этих систем.

### Введение

Генеративные модели выделяются своей способностью создавать новые данные, соответствующие заданному образцу или распределению. Применение этих технологий варьируется от синтеза речи до генерации текстов и изображений. По мере развития этих технологий возникают вопросы о потенциальном сознании искусственных систем, что ставит перед исследователями задачу тестирования AI на наличие сознательных свойств.

### История

Первые предположения о возможности создания сознательных машин появились в середине 20-го века, вскоре после предложения Аланом Тьюрингом его знаменитого теста. Исследования в этой области получили новый импульс с развитием глубокого обучения и генеративных моделей, что привело к экспериментам с использованием этих технологий для имитации человеческого поведения и тестирования потенциального сознания[1][2].

### Философские аспекты

Философские аспекты затрагивают глубокие вопросы о природе сознания, его возможности существования вне биологических систем и этических последствиях такого существования. Дебаты в этой области касаются определения сознания, критериев его наличия у ИИ и возможных прав и обязанностей сознательных систем[3].

### Этические аспекты

Этика тестирования AI на наличие сознания охватывает вопросы прав и обязанностей искусственного интеллекта, этические проблемы создания сознательных систем и вопросы ответственности за действия ИИ[4].

### Когнитивные науки и AI

Интеграция когнитивных наук и искусственного интеллекта открывает новые перспективы для понимания и моделирования человеческого разума. Это мультидисциплинарное взаимодействие помогает улучшить алгоритмы машинного обучения и разработать более продвинутые системы искусственного интеллекта[5].

### Существующие методы тестирования

Среди методов тестирования наличия сознания у ИИ выделяются Тест Тьюринга, тест на основе интегрированной информационной теории и тест на искусственное сознание. Эти методы направлены на выявление способностей ИИ к самосознанию, мышлению и взаимодействию на уровне, сопоставимом с человеческим[6].

### Примеры тестов

Тесты на сознание ИИ включают Тест Тьюринга, Теорию интегрированной информации и критерии нейробиологов, каждый из которых предлагает уникальный подход к оценке потенциального сознания искусственных систем[7].

### Вызовы и перспективы развития

Возможности развития тестирования сознания в ИИ включают увеличение размера моделей, использование высокопроизводительных вычислений, инновационные технологии и подходы. Эти перспективы предлагают захватывающие возможности для дальнейшего исследования сознания в искусственных системах[8].

### Заключение

Исследование сознания в генеративных моделях ИИ стоит на пересечении науки, философии и этики. Продолжающиеся дебаты и исследования в этой области способствуют глубокому пониманию сознания и его роли в развитии интеллектуальных систем. В то же время, они подчеркивают важность ответственного использования ИИ и необходимость учитывать этические аспекты при его разработке и применении[9].

### Ссылки

1. https://academic.oup.com/book/33540/chapter/287907169
2. https://new-science.ru/kak-issledovateli-mogut-opredelit-obrel-li-ii-soznanie/
3. https://www.nanonewsnet.ru/news/2023/neirobiologi-predlozhili-kriterii-nalichiya-soznaniya-u-ii
4. https://blog.clientpro.ai/articles/b/testing-ai-consciousness-3-key-methods
5. https://nauchniestati.ru/spravka/neskolko-slov-o-soznanii-ii/
6. https://qaa-engineer.ru/kakie-metody-ispolzuyutsya-dlya-testirovania-iskusstvennogo-intellekta/
7. https://blog.clientpro.ai/articles/b/testing-ai-consciousness-3-key-methods
8. https://aipromptopus.com/ai-news/ai-consciousness/
9. https://new-science.ru/kak-issledovateli-mogut-opredelit-obrel-li-ii-soznanie/